<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_nl_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-62mmjthq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-62mmjthq
  Resolved https://github.com/huggingface/transformers to commit b074461ef0f54ce37c5239d30ee960ece28d11ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [14]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [15]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [16]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [17]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 173MiB/s]


In [18]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [19]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl")
    return result["text"]

In [20]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [21]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [22]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.2, Precision: 0.8333333333333334, Recall: 0.8, F1: 0.816326530612245
Processed 2 files, WER: 0.5769230769230769, Precision: 0.5517241379310345, Recall: 0.64, F1: 0.5925925925925927
Processed 3 files, WER: 0.3181818181818182, Precision: 0.65625, Recall: 0.6176470588235294, F1: 0.6363636363636364
Processed 4 files, WER: 0.45, Precision: 0.5714285714285714, Recall: 0.6, F1: 0.5853658536585366
Processed 5 files, WER: 0.6842105263157895, Precision: 0.4117647058823529, Recall: 0.4117647058823529, F1: 0.4117647058823529
Processed 6 files, WER: 0.6428571428571429, Precision: 0.5333333333333333, Recall: 0.6153846153846154, F1: 0.5714285714285715
Processed 7 files, WER: 0.34210526315789475, Precision: 0.6363636363636364, Recall: 0.6176470588235294, F1: 0.6268656716417911
Processed 8 files, WER: 0.3333333333333333, Precision: 0.8, Recall: 0.7058823529411765, F1: 0.7500000000000001
Processed 9 files, WER: 0.4, Precision: 0.6111111111111112, Recall: 0.6111111111111112, F1: